In [1]:
from pathlib import Path
import sys
root = Path.cwd().parent
sys.path.insert(1, str(root))
%load_ext autoreload
%autoreload 2

In [2]:
import cv2
import dlib
import face_recognition
import numpy as np
from pathlib import Path
from sklearn.svm import SVC
from tensorflow.keras.models import load_model
from sklearn.preprocessing import Normalizer
import warnings
warnings.filterwarnings("ignore")

In [3]:
from appconfig.config import MODEL_DIR

In [4]:
detector = dlib.get_frontal_face_detector()

In [5]:
import pickle
with open('SVM_be_cham.sav', 'rb') as f:
    clf = pickle.load(f)
    
with open('decoder_be_cham.pickle', 'rb') as f:
    decoder = pickle.load(f)
    
face_model = load_model(MODEL_DIR)

In [6]:
clf

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [7]:
def get_face_boundary(frame, upsample = 0):
    rects = detector(frame, upsample)
    rects_list = []
    if len(rects) > 0:   
        for rect in rects:
            rects_list.append(rect)
        return rects_list
    else:
        return

def draw_rectangle_around_faces(frame, rect_list):
    for rect in rect_list:
        left = rect.left()
        top = rect.top()
        right = rect.right()
        bottom = rect.bottom()
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
    return 

def crop_face(frame, rect_list):
    faces = []
    for rect in rect_list:
        left = rect.left()
        top = rect.top()
        right = rect.right()
        bottom = rect.bottom()
        faces.append(frame[top: bottom, left: right])
    return faces

def standardize(pixels):
    mean, std = pixels.mean(), pixels.std()
    pixels_std = (pixels - mean) / std
    return pixels_std

def get_features(face):
    array = np.asarray(face, dtype=np.float32)
    array = cv2.resize(array, (160,160))
    array = standardize(array)
    array = np.expand_dims(array, axis=0)
    yhat = face_model.predict(array)
    yhat = Normalizer().transform(yhat)
    return yhat

def preprocess_frame(faces, rect_list):
    x_test = np.zeros(shape=(len(faces), 128))
    i = 0
    for face in faces:
        features = get_features(face)
        x_test[i] = features

    
    name = clf.predict(x_test)
    name = decoder.inverse_transform(name)
    return zip(name, rect_list)

def run_facedetector_camera():
    cap = cv2.VideoCapture(0)

    i = 0
    font = cv2.FONT_HERSHEY_SIMPLEX 
    # fontScale 
    fontScale = 1

    # Red color in BGR 
    color = (0, 0, 255) 

    # Line thickness of 2 px 
    thickness = 2
    
    while (True):
        # Capture frame-by-frame
        _, frame = cap.read()
        if get_face_boundary(frame, upsample=0):
            rect_list = get_face_boundary(frame, upsample=0)
            draw_rectangle_around_faces(frame, rect_list)

            faces = crop_face(frame, rect_list)
            try:
                for name, rect in preprocess_frame(faces, rect_list):
                    left = rect.left()
                    right = rect.right()
                    bottom = rect.bottom()
                    cv2.putText(frame, name, ((left + right) // 2, bottom + 20), font, fontScale,  color, thickness, cv2.LINE_AA, False)
            except:
                pass

        # Display the resulting frame
        cv2.imshow('Image', frame)
            
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()
    
    return

In [8]:
run_facedetector_camera()

In [7]:
run_facedetector_camera()